In [4]:
import os
import pandas as pd
#import pyreadstat
import glob
from functools import wraps
from time import time
import savReaderWriter as spss

In [2]:
#Además de los decimales, del word, saca los valores que sirven la checkear que la carga es correcta
#Automatizando eso en una función, if True, then print('los datos están bien')
#Pide a Laura que te pase las tablas dinámicas
#INtenta construir una serie temporal para análisis
#Reproduce los análisis que han hecho en el WORD del estudio

cruce de variables dentro de la encuesta.

    Tasa Paro - Nªocupados (por trimestres) (En este caso la fuente es la EPA)
    Índice de pobreza según OCDE (anual)
    Gasto en agua y carga con respecto a ingresos
    Ingresos anuales - Gasto agua (abastecimiento y saneamiento / solo abastecimiento)
    Consumo agua (persona/litro) - Gasto agua (abastecimiento y saneamiento / solo abastecimiento

In [7]:
def timer(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print('Elapsed time: {}'.format(end-start))
        return result
    return wrapper

#Funciones GASTOS
@timer
def get_tablas_gastos(base_dir):
    for file in os.listdir(base_dir):
        file_path = f'{base_dir}/{file}'
        if '.csv' not in file_path:
            with spss.SavReader(file_path, ioUtf8 = True, rawMode=True) as reader:
                df = pd.DataFrame(reader.all(), columns = [s for s in reader.header])
                df = df.apply(lambda x: x.astype(float)/100 if x.name not in ['ANOENC', 'NUMERO','CODIGO', 'NACTOR'] else x)
                df = df.apply(lambda x: x.astype(int) if x.name in ['ANOENC', 'NUMERO'] else x)
                df = df.apply(lambda x: x/1000000 if x.name in ['NACTOR'] else x)
                df.to_csv(base_dir+'/gastos_I_'+ str(df.iloc[0][0]) + '.csv')

@timer
def load_tablas_gastos(base_dir):
    all_files = glob.glob(base_dir + "/*.csv")
    
    li = []
    for fname in all_files:
        df = pd.read_csv(fname, index_col=None, header=0)
        li.append(df)

    dframe = pd.concat(li, axis=0, ignore_index=True) 
    dframe.to_csv('gastos_serie_completa.csv')
    


#Funciones HOGAR
@timer
def get_tablas_hogar(base_dir):
    for file in os.listdir(base_dir):
        file_path = f'{base_dir}/{file}'
        if '.csv' not in file_path:
            with spss.SavReader(file_path, ioUtf8 = True, rawMode=True) as reader:
                df = pd.DataFrame(reader.all(), columns = [s for s in reader.header])
                df = df.apply(lambda x: x.astype(int) if x.name in ['ANOENC', 'NUMERO'] else x)
                df.to_csv(base_dir+'/hogar_'+ str(df.loc[0, "ANOENC"]) + '.csv',  index=False)

@timer
def load_tablas_hogar(base_dir):
    all_files = glob.glob(base_dir + "/*.csv")
    
    li = []
    for fname in all_files:
        df = pd.read_csv(fname, index_col=None, header=0)
        li.append(df)

    dframe = pd.concat(li, axis=0, ignore_index=True) 
    dframe.to_csv('hogar_serie_completa.csv')

def validity(df_gastos, df_hogar):
    return df_gastos.shape == df_hogar.shape
    
#Funciones MIEMBROS

In [ ]:
ruta = "/home/fserrey/gitrep/ACA_project/docs_table/gastos_tablas"
get_tablas_gastos(ruta)

In [7]:
ruta = "/home/fserrey/gitrep/ACA_project/docs_table/gastos_tablas"
load_tablas_gastos(ruta)

/home/fserrey/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Elapsed time: 576.0942797660828


In [10]:
file = pd.read_csv('../../../../ACA_project/docs_table/gastos_tablas/gastos_I_2010.csv')
file.columns

Index(['Unnamed: 0', 'ANOENC', 'NUMERO', 'CODIGO', 'GASTO', 'PORCEN',
       'CANTIDAD', 'GASTMON', 'GASTNOM1', 'GASTNOM2', 'GASTNOM3', 'GASTNOM4',
       'GASTNOM5', 'NACTOR'],
      dtype='object')

In [31]:
all_files = glob.glob('../../../../ACA_project/docs_table/gastos_tablas' + "/*.csv")
match = [file.split("_")[-1] for file in all_files]
dates = sorted([int(dat.split('.')[0]) for dat in match])


[2006, 2007, 2008, 2009, 2010, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


In [3]:
base_dir = '../../../../ACA_project/docs_table/gastos_tablas'
#df = pd.concat(map(pd.read_csv, glob.glob(base_dir + "/*.csv")))
all_files = glob.glob(base_dir + "/*.csv")
    
li = []
for fname in all_files:
    df = pd.read_csv(fname, index_col=None, header=0)
    li.append(df)

dframe = pd.concat(li, axis=0, ignore_index=True)

/home/fserrey/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


MemoryError: Unable to allocate array with shape (2, 29000065) and data type float64

In [ ]:
gastos_tabla.columns

In [8]:
ruta = "/home/fserrey/gitrep/ACA_project/docs_table/hogar_tablas"

get_tablas_hogar(ruta)

Elapsed time: 51.95612096786499


In [ ]:
load_tablas_hogar(ruta)

In [10]:
hogar_tabla = pd.read_csv('hogar_serie_completa.csv')

In [ ]:
hogar_tabla.head(2)

In [11]:
hogar_tabla.ANOENC.value_counts()

2009    22346
2010    22203
2008    22077
2007    21542
2006    19435
Name: ANOENC, dtype: int64

In [ ]:
validity(gastos_tabla, hogar_tabla)

In [ ]:
gastos_tabla.shape

In [ ]:
hogar_tabla.shape

In [ ]:
list(hogar_tabla.columns)

In [ ]:
gastos_tabla.columns

In [ ]:
gastos_tabla.head(1)

In [2]:
all_gastos = glob.glob('../../../../ACA_project/docs_table/gastos_tablas/*.csv')
all_hogar = glob.glob('../../../../ACA_project/docs_table/hogar_tablas/*.csv')


In [3]:
li = []
for gasto in sorted(all_gastos):
    some_values = list(range(4511,4551))
    df = pd.read_csv(gasto)
    df['CODIGO'] = df['CODIGO'].apply(lambda x: x.split("'")[1])
    df['CODIGO'] = df['CODIGO'].apply(lambda x: int(x.split(" ")[0]) if str(x.split(" ")[0]).isdigit() else None)
    df = df.loc[df['CODIGO'].isin(some_values)]
    li.append(df)

In [4]:
lo = []
for hogar in sorted(all_hogar):
    df = pd.read_csv(hogar)
    df['CCAA'] = df['CCAA'].apply(lambda x: x.split("'")[1])
    df['CCAA'] = df['CCAA'].apply(lambda x: int(x.split(" ")[0]) if str(x.split(" ")[0]).isdigit() else None)
    lo.append(df)

In [1]:
final = []
for gastos in li:
    for hogares in lo:
        new_df = pd.merge(gastos, hogares, how='left', on=['ANOENC','NUMERO'])
        final.append(new_df)

NameError: name 'li' is not defined

In [ ]:
dframe = pd.concat(final, axis=0, ignore_index=True)

In [35]:
gastos_tabla = pd.read_csv('../../../../ACA_project/docs_table/gastos_tablas/gastos_I_2016.csv')
hogar_tabla = pd.read_csv('/home/fserrey/gitrep/ACA_project/docs_table/hogar_tablas/hogar_2016.csv')
gastos_tabla.head()

,Unnamed: 0,ANOENC,NUMERO,CODIGO,GASTO,PORCENDES,PORCENIMP,CANTIDAD
0,0,2016,1,b'01111 ',37.22,100.00,100.00,-1.797693e+306
1,1,2016,1,b'01112 ',14.79,100.00,100.00,-1.797693e+306
2,2,2016,1,b'01113 ',113964.81,0.50,83.00,4.586956e+04
3,3,2016,1,b'01114 ',54614.37,0.69,0.69,5.811310e+03
4,4,2016,1,b'01115 ',73.85,100.00,100.00,-1.797693e+306


In [32]:
gastos_tabla.CODIGO[0]

"b'01111   '"

In [36]:
some_values = list(range(4511,4551))
gastos_tabla['CODIGO'] = gastos_tabla['CODIGO'].apply(lambda x: x.split("'")[1])
gastos_tabla['CODIGO'] = gastos_tabla['CODIGO'].apply(lambda x: int(x.split(" ")[0]) if str(x.split(" ")[0]).isdigit() else None)

hogar_tabla['CCAA'] = hogar_tabla['CCAA'].apply(lambda x: x.split("'")[1])
hogar_tabla['CCAA'] = hogar_tabla['CCAA'].apply(lambda x: int(x.split(" ")[0]) if str(x.split(" ")[0]).isdigit() else None)

In [28]:

gastos_tabla.CODIGO[0]

1111.0

In [37]:
gastos_filt = gastos_tabla.loc[gastos_tabla['CODIGO'].isin(some_values)]
new_df = pd.merge(gastos_filt, hogar_tabla, how='left', on=['ANOENC','NUMERO'])

In [38]:
new_df

,Unnamed: 0_x,ANOENC,NUMERO,CODIGO,GASTO,PORCENDES,PORCENIMP,CANTIDAD,Unnamed: 0_y,CCAA,...,FUENPRIN,FUENPRINRED,IMPEXAC,INTERIN,NUMPERI,COMIMH,COMISD,COMIHU,COMIINV,COMITOT
0,83,2016,1,4511.0,356639.47,0.00,0.00,1438148.67,0,7,...,b' 1 ',1.0,1225.0,b'03 ',2.0,8.0,0.0,0.0,0.0,8.0
1,84,2016,1,4521.0,160487.76,0.00,0.00,122149.02,0,7,...,b' 1 ',1.0,1225.0,b'03 ',2.0,8.0,0.0,0.0,0.0,8.0
2,149,2016,2,4511.0,911914.07,0.00,0.00,5104408.53,1,12,...,b' 7 ',3.0,676.0,b'02 ',2.0,38.0,0.0,0.0,0.0,38.0
3,150,2016,2,4521.0,641263.93,0.00,0.00,1199275.29,1,12,...,b' 7 ',3.0,676.0,b'02 ',2.0,38.0,0.0,0.0,0.0,38.0
4,300,2016,3,4511.0,1669142.21,0.03,0.03,6507827.99,2,13,...,b' 2 ',2.0,5868.0,b'08 ',4.0,89.0,0.0,0.0,10.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55050,2285213,2016,22010,4511.0,155748.66,0.00,0.00,945382.14,22009,3,...,b' 3 ',3.0,1633.0,b'04 ',1.0,56.0,0.0,0.0,8.0,64.0
55051,2285214,2016,22010,4512.0,73133.34,0.00,0.00,194099.06,22009,3,...,b' 3 ',3.0,1633.0,b'04 ',1.0,56.0,0.0,0.0,8.0,64.0
55052,2285215,2016,22010,4531.0,117351.31,0.00,0.00,176026.96,22009,3,...,b' 3 ',3.0,1633.0,b'04 ',1.0,56.0,0.0,0.0,8.0,64.0
55053,2285276,2016,22011,4511.0,119465.11,0.00,0.00,488437.50,22010,16,...,b' 2 ',2.0,2500.0,b'06 ',2.0,75.0,0.0,0.0,0.0,75.0
